In [1]:
from flask import Flask, jsonify, request

app = Flask(__name__)

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

df = pd.read_csv('test.csv')

In [2]:
from sentence_transformers import SentenceTransformer
model1 = SentenceTransformer('distilbert-base-nli-mean-tokens')

c:\Users\Rajesh\Desktop\Project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def similarity(question, lst):
    sentence_vector = model1.encode([question])[0]
    sentences_vectors = model1.encode(lst)

    # Compute the cosine similarity between the sentence vector and each of the sentences vectors
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = [cosine_similarity([sentence_vector], [sentences_vector])[0][0] for sentences_vector in sentences_vectors]

    # Find the index of the most similar sentence
    most_similar_index = similarities.index(max(similarities))
    return most_similar_index

In [4]:
import gensim
from gensim import similarities
def sim(sentence):
    # Define the sentence and the list of sentences
    sentence = sentence
    sentences = list(df['Answer'])

    # Create a dictionary from the sentences
    dictionary = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(s) for s in sentences])

    # Convert the sentences into vectors using TF-IDF
    tfidf_corpus = [dictionary.doc2bow(s) for s in [gensim.utils.simple_preprocess(s) for s in sentences]]
    tfidf_model = gensim.models.TfidfModel(tfidf_corpus)
    index = similarities.MatrixSimilarity(tfidf_model[tfidf_corpus], num_features=len(dictionary))

    # Convert the query sentence to TF-IDF vector
    query_vec = tfidf_model[dictionary.doc2bow(gensim.utils.simple_preprocess(sentence))]

    # Calculate similarities
    sims = index[query_vec]

    # Find the index of the highest similarity
    max_index = sims.argmax()

    # Return the sentence with the highest similarity
    return max_index

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [6]:
# @app.route('/img', methods=['GET'])
# def img():
#     paragraph =  str(request.args['text'])
#     doc = nlp(paragraph)
#     d = {}
#     l=[]
#     sentences = [sent.text for sent in doc.sents]
#     for sentence in sentences:
#         # Your data generation logic
#         i = similarity(sentence, list(df['Answer']))
#         l.append(df['Path'][i])
#     d['img'] = l
#     return d


In [7]:
@app.route('/img', methods=['GET'])
def img():
    paragraph =  str(request.args['text'])
    doc = nlp(paragraph)
    d = {}
    l=[]
    sentences = [sent.text for sent in doc.sents]
    for sentence in sentences:
        # Your data generation logic
        i = sim(sentence)
        l.append(df['Path'][i])
    d['img'] = l
    return d


In [8]:
if  __name__ == '__main__':
    app.run(port=5002)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 12:46:47] "GET /img?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biological%20system%20on%20Earth.%20Several%20microorganisms%20arebeneficial%20and%20contribute%20to%20human%20welfare.%20Microbes%20are%20presenteverywhere%20–%20in%20soil,%20water,%20air%20and%20within%20bodies%20of%20animals%20and%20plants.%20Microbes%20like%20bacteria%20and%20fungi%20can%20be%20grown%20on%20nutritive%20media%20to%20form%20colonies%20which%20can%20be%20visibly%20seen.%20Some%20of%20the%20microbes%20useful%20to%20human%20welfare%20are%20discussed%20here HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:20:01] "GET /img?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biological%2

In [ ]:
df.head()

In [ ]:
def img(paragraph):
    # paragraph =  str(request.args['text'])
    doc = nlp(paragraph)
    d = {}
    l=[]
    sentences = [sent.text for sent in doc.sents]
    for sentence in sentences:
        # Your data generation logic
        i = similarity(sentence, list(df['Answer']))
        l.append(df['Path'][i])
    d['img'] = l
    return d

In [ ]:
img("Microbes are not only used for commercial and industrial production of alcohol, but also used for production of chemicals like organic acids and enzymes. Examples of organic acid producers are Aspergillus niger for citric acid, Acetobacter aceti for acetic acid, Rhizopus oryzae for fumaric acid, Clostridium butyricum for butyric acid and Lactobacillusfor lactic acid. Yeast (Saccharomyces cerevisiae) and bacteria are used for commercial production of enzymes. Lipases are used in detergent formulations and are used for removing oily stains from the laundry. Bottled juices are clarified by the use of pectinase,protease and cellulase. Rennet can also be used to separate milk into solid curds for cheese making. Streptokinase produced by the bacterium Streptococcus and genetically engineered Streptococci are used as \u201cclot buster\u201d for removing clots from the blood vessels of patients who have undergone myocardial infarction")

{'img': ['41.png',
  '131.png',
  '36.png',
  '43.png',
  '20.png',
  '125.png',
  '75.png']}

In [ ]:
import os 
import random 

if (random(0,6))